# **Projeto 1 - Ciência dos Dados**  


### **Equipe**
- **Denis Alonso**  
- **Felipe Menke**  
- **Gabriel Correia**  

___
Carregando algumas bibliotecas:

In [93]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [94]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\gabriel\Documents\insper\terceiro_semestre\ccdados2\proj1\Projeto1-Cdados-Insper


___

### Carregando a base de dados com as mensagens dos seus arquivos. <br> Tire o `#` do início da linha de código condizente ao caso escolhido para o projeto 1.

In [95]:
# SE SEU PROJETO USA OS DADOS SOBRE "Airline Passenger Reviews"
train = pd.read_csv('dados_treino_ate_TRIO_FelipeMenke.csv')
test = pd.read_csv('dados_teste_ate_TRIO_FelipeMenke.csv')

In [96]:
# SE SEU PROJETO USA OS DADOS SOBRE "ChatGPT Classification"
#train = pd.read_csv('dados_treino_so_DUPLA_'+nome+'.csv')
#test = pd.read_csv('dados_teste_so_DUPLA_'+nome+'.csv')

In [97]:
train.head()

,Review,Target
0,Singapore to Jakarta. 9 years since I last to...,Passive
1,Norwegian Long Haul. OSL-LGW. Flight DY1310. 1...,Promoter
2,Guangzhou to Paris. I have paid for inflight...,Promoter
3,Vientiane to Melbourne via Bangkok. A very en...,Promoter
4,I travelled Manchester - Sydney return on 777-...,Passive


In [98]:
test.head()

,Review,Target
0,Budapest to Warsaw. The ground crew was very ...,Passive
1,Bergerac to Stansted. Just come off a particul...,Detractor
2,BRU-LIS. Flight left a few minutes late aircra...,Promoter
3,Fleet of A319 and A320 in this route. Fast che...,Passive
4,4 Nov Houston-Doha QR714 and 15 Nov Doha-Houst...,Passive


___
# Classificador Automático (Boot)


Neste projeto, trabalhamos com um conjunto de dados de **avaliações de passageiros de companhias aéreas**. Essas avaliações trazem a perspectiva dos clientes sobre a qualidade do serviço oferecido, podendo evidenciar fatores como conforto, pontualidade, atendimento e muito mais. Para analisar a satisfação desses passageiros, cada avaliação (ou *review*) recebeu um dos seguintes rótulos, alinhados à metodologia **Net Promoter Score (NPS)**:

- **Detractor**: Clientes insatisfeitos, que atribuem notas entre 0 e 6 e tendem a desincentivar a utilização do serviço.  
- **Promoter**: Clientes altamente satisfeitos, que geralmente atribuem notas 9 ou 10, e tornam-se promotores fiéis da marca.  
- **Other**: Avaliações que não se encaixam completamente como insatisfeitas ou promotoras (notas 7 e 8), representando um meio-termo entre os dois extremos.

A classificação automática dessas avaliações é fundamental para que as companhias aéreas possam **identificar tendências** de satisfação ou insatisfação, priorizar melhorias estratégicas no atendimento e, por consequência, **tomar decisões baseadas em dados**. Por meio do **Classificador Naive-Bayes**, buscamos construir um modelo que atribua, de forma confiável, o rótulo adequado a cada avaliação, a partir do texto livre escrito pelos passageiros.


___
## Montando SEU Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

Nesta etapa, construímos o nosso classificador Naive-Bayes a partir das **mensagens da planilha de Treinamento**. O processo envolve:

1. **Limpeza do Texto**  
   - A função `cleanup(text)` remove quebras de linha (`\n`), caracteres especiais (`\r`), pontuação e espaços em branco em excesso. Dessa forma, cada *review* é transformado em uma sequência de palavras mais padronizada, facilitando os cálculos de frequência.

In [99]:
import re 

def cleanup(text):
    
    text = text.replace('\n', ' ').replace('\r', ' ')    
    text = re.sub(r'\s+', ' ', text)
    punctuation = '[´"!-.:?;$'']' 
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    
    return text_subbed

**Todas as Avaliações**

Reunimos todas as avaliações do conjunto de **treinamento** em uma única lista de textos e, em seguida, fazemos a limpeza com a função `cleanup()` para remover pontuações e caracteres especiais. Depois:

1. **Unificamos** todas as avaliações em uma string única, tornando mais fácil trabalhar sobre o texto como um todo.  
2. **Convertemos** tudo para minúsculas (lowercase), garantindo padronização.  
3. **Separamos** cada palavra (split) e transformamos em um objeto `pd.Series` para facilitar a contagem.  
4. **Obtemos** a contagem final de palavras (`value_counts()`), resultando em quantas vezes cada termo aparece no conjunto de treinamento.

Essas estatísticas nos darão uma visão geral do **vocabulário** do modelo, incluindo o volume total de termos e as palavras mais frequentes.

In [100]:
#Descobrindo a quantidade total de palavras
dic_total = train.loc[:, "Review"]  
palavras_totais = dic_total.tolist()  
palavras_totais = " ".join(palavras_totais)  
palavras_totais = cleanup(palavras_totais).lower()  
palavras_totais = palavras_totais.split()  
palavras_totais = pd.Series(palavras_totais)  
qtd_palavras_totais = palavras_totais.value_counts()  
qtd_palavras_totais = qtd_palavras_totais.sum() 


**Dados - Avaliações Perfil Passivo**


Aqui, extraímos todas as *reviews* classificadas como **Passive** e:

1. **Reunimos** essas avaliações em uma lista, convertendo-as em uma única string.
2. **Limpamos** o texto utilizando a função `cleanup`, removendo pontuações e caracteres indesejados, além de converter tudo para letras minúsculas.
3. **Separamos** as palavras (`split`) para poder contabilizá-las.
4. **Calculamos**:
   - O **número total** de palavras encontradas em *reviews* passivas.
   - A **probabilidade a priori** de um texto ser *Passive*, dividindo o total de palavras passivas pelo total de palavras de **todas** as avaliações de treinamento.
   - As **frequências relativa** e **absoluta** de cada palavra específica dentro das avaliações rotuladas como Passive.

Esse conjunto de contagens e probabilidades alimentará posteriormente o modelo Naive-Bayes, contribuindo para classificar novas *reviews* como pertencentes (ou não) a essa categoria.

E o mesmo será feito para Detractor e Promoter

In [101]:
#PASSIVO

dic_passivos = train.loc[train["Target"]=="Passive","Review"]
#Transformando em Lista
frases_passivos = dic_passivos.tolist()

#Juntando as frases,Limpando (caracteres especiais, e diminuindo as letras)
frases_passivos = " ".join(frases_passivos)
frases_passivos = cleanup(frases_passivos).lower()
frases_passivos = frases_passivos.split()

#Probabilidade de ser Passivo
qtd_palavras_passivos = pd.Series(frases_passivos).value_counts()
qtd_palavras_passivos = qtd_palavras_passivos.sum()

Prob_Passivo = qtd_palavras_passivos/qtd_palavras_totais

#Criando Frequencia Relativa das Palavras

freq_rel_passivos = pd.Series(frases_passivos).value_counts(True)
freq_abs_passivos = pd.Series(frases_passivos).value_counts()

**Dados - Avaliações Perfil Detrator**

In [102]:
#DETRACTOR

dic_detrator = train.loc[train["Target"]=="Detractor","Review"]

#Transformando em Lista
frases_detractor = dic_detrator.tolist()
#Juntando as frases,Limpando (caracteres especiais, e diminuindo as letras)
frases_detractor = " ".join(frases_detractor)
frases_detractor = cleanup(frases_detractor).lower()
frases_detractor = frases_detractor.split()

#Probabilidade de ser Detrator
qtd_palavras_detrator = pd.Series(frases_detractor).value_counts()
qtd_palavras_detrator = qtd_palavras_detrator.sum()

Prob_Detrator = qtd_palavras_detrator/qtd_palavras_totais

#Criando Frequencia Relativa das Palavras

freq_rel_detrator = pd.Series(frases_detractor).value_counts(True)
freq_abs_detrator = pd.Series(frases_detractor).value_counts()

**Dados - Avaliações Perfil Promoter**

In [103]:
#PROMOTER

dic_promoter = train.loc[train["Target"]=="Promoter","Review"]

#Transformando em Lista
frases_promoter = dic_promoter.tolist()

#Juntando as frases,Limpando (caracteres especiais, e diminuindo as letras)
frases_promoter = " ".join(frases_promoter)
frases_promoter = cleanup(frases_promoter).lower()
frases_promoter = frases_promoter.split()

#Probabilidade de ser Detrator
qtd_palavras_promoter = pd.Series(frases_promoter).value_counts()
qtd_palavras_promoter = qtd_palavras_promoter.sum()

Prob_Promoter = qtd_palavras_promoter/qtd_palavras_totais

#Criando Frequencia Relativa das Palavras

freq_rel_promoter = pd.Series(frases_promoter).value_counts(True)
freq_abs_promoter = pd.Series(frases_promoter).value_counts()
print(freq_rel_promoter)

the             0.049208
and             0.037813
to              0.029852
was             0.026083
a               0.021977
                  ...   
longyearbyen    0.000011
svalbard        0.000011
osl/lyr         0.000011
â£175           0.000011
mandarin        0.000011
Name: proportion, Length: 6755, dtype: float64


Nesta etapa, construímos o **vocabulário** do modelo, que consiste em uma lista contendo todas as palavras únicas encontradas nas avaliações do conjunto de treinamento. O processo ocorre da seguinte forma:

1. **Iteramos sobre todas as palavras** encontradas nas avaliações.  
2. **Adicionamos ao vocabulário** apenas palavras que ainda não foram registradas, garantindo que cada termo apareça apenas uma vez.  
3. **Armazenamos a contagem total de palavras únicas**, ou seja, o tamanho do vocabulário, que será usado posteriormente nos cálculos do modelo.  

Este vocabulário é essencial para o **classificador Naive-Bayes**, pois será a base para determinar a frequência e a probabilidade de cada palavra em relação às classes de avaliações (*Detractor*, *Promoter* e *Other*).

In [104]:
#Criando VOCABULARIO
#Adicionando todas as palavras presentes nas frases uma unica vez:
vocabulario = []

for i in palavras_totais:
    if i not in vocabulario:
        vocabulario.append(i)

print(vocabulario)

qtd_vocabulario = len(vocabulario)

['singapore', 'to', 'jakarta', '9', 'years', 'since', 'i', 'last', 'took', 'airlines', 'online', 'checkin', 'process', 'easy', 'managed', 'call', 'the', 'hotline', 'change', 'my', 'seat', 'get', 'an', 'exit', 'row', 'with', 'parents', 'which', 'was', 'done', 'easily', 'sq', 'did', 'even', 'bother', 'send', 'us', 'sms', 'warn', 'about', 'french', 'strikes', 'blocking', 'roads', 'and', 'go', 'early', 'airport', 'subsequently', 'very', 'good', 'point', 'find', 'quality', 'of', 'service', 'onboard', 'below', 'that', 'few', 'ago', 'food', 'standard', 'bland', 'staff', 'hardskills', 'though', 'responsive', 'efficient', 'professional', 'but', 'could', 'be', 'better', 'they', 'talk', 'fast', 'robotic', 'gestures', 'more', 'smiling', 'on', 'our', 'inbound', 'leg', 'ran', 'out', 'one', 'option', 'left', 'disappointed', 'seats', 'boeing', '777300er', 'larger', 'comfortable', 'compared', 'other', 'premium', 'for', 'outbound', 'jakartasingapore', 'asked', 'if', 'we', 'wanted', 'take', 'earlier', 'f

___
## Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

Testamos a eficiência do **classificador Naive-Bayes** utilizando a **base de testes**. O processo segue as seguintes etapas:

1. **Preparação dos Dados de Teste**  
   - Extraímos os rótulos reais das avaliações (`Target_test`).  
   - Extraímos o conteúdo textual das avaliações (`Review_test`).  
   - Criamos um dicionário para armazenar os resultados da classificação.

2. **Aplicação do Modelo com Suavização de Laplace**  
   - Para cada avaliação, aplicamos a mesma **limpeza e processamento** utilizados na base de treinamento.  
   - Inicializamos as probabilidades para cada categoria (*Detractor*, *Promoter* e *Passive*).  
   - Para cada palavra da avaliação:
     - Se a palavra estiver no vocabulário de uma das classes, multiplicamos sua frequência normalizada na classe correspondente.
     - Caso contrário, aplicamos a **suavização de Laplace**, garantindo que nenhuma palavra desconhecida zere a probabilidade de uma classe.

3. **Predição da Categoria**  
   - Após calcular as probabilidades para cada classe, escolhemos a que apresentar o maior valor.  
   - O rótulo predito é armazenado no dicionário de resultados junto ao rótulo real.

4. **Criação do DataFrame de Resultados**  
   - Convertamos o dicionário de predições em um **DataFrame pandas**, facilitando a análise dos acertos e erros do modelo.

Este processo permite avaliar o desempenho do classificador e, posteriormente, calcular métricas como **acurácia, precisão, recall e matriz de confusão** para validar a eficácia do modelo.



In [105]:
##Codigo da suavização de Laplace
dic_final = {}
Target_test = test.loc[:, "Target"]
Review_test = test.loc[:, "Review"]

for i in range(len(test)):
    tipo = Target_test.iloc[i]  
    frase = cleanup(Review_test.iloc[i]).lower().strip()
    frase = " ".join(frase.split())  

    probDetratordadoFrase = 1
    probPassivodadoFrase = 1
    probPromoterdadoFrase = 1

    for palavra in frase.split():
        palavra = palavra.lower().strip()


        if palavra in frases_promoter:
            probPromoterdadoFrase *= (freq_abs_promoter[palavra]+ 1)/(qtd_palavras_promoter + qtd_vocabulario)  * 1000
        else:
            probPromoterdadoFrase *=(1/(qtd_palavras_promoter + qtd_vocabulario))  * 1000
            
        if palavra in frases_detractor:
            probDetratordadoFrase *= (freq_abs_detrator[palavra]+ 1)/(qtd_palavras_detrator + qtd_vocabulario)   * 1000
        else:
            probDetratordadoFrase *= (1/(qtd_palavras_detrator + qtd_vocabulario)) * 1000

        if palavra in frases_passivos:
            probPassivodadoFrase *= (freq_abs_passivos[palavra]+ 1)/(qtd_palavras_passivos + qtd_vocabulario)   * 1000
        else:
            probPassivodadoFrase *= (1/(qtd_palavras_passivos + qtd_vocabulario))  * 1000

    maior = max(probDetratordadoFrase, probPromoterdadoFrase, probPassivodadoFrase)
    print( probDetratordadoFrase, probPromoterdadoFrase, probPassivodadoFrase)

    if maior == probDetratordadoFrase:
        final = "Detractor"
    elif maior == probPromoterdadoFrase:
        final = "Promoter"
    else:
        final = "Passive"

    dic_final[i] = (tipo, final)

import pandas as pd
df_resultado = pd.DataFrame.from_dict(dic_final, orient="index", columns=["Target", "Predição"])

df_resultado

9666.347548791027 27438954871.118313 768463785.0879236
9.620232611699435e+26 2.8345211907256368e+17 4.071239183355169e+25
37428041.318441644 846222289761031.5 432087194653750.06
13.551515473649344 648180.6029658039 122847470.16127603
1086319.2780372838 72945020530.08113 22290690903462.957
4194203218136.0205 8.141642868171042e+19 1.0738709280431299e+18
66923437.043524444 43107413306.25221 31630579402.854965
1091427126009235.6 6.709293678452009e+17 2.1258697383200417e+25
4.1723560862387976e+16 1705300974904.6577 5.660241364898573e+16
27432675.939307764 81893485.18352792 370314341.70884967
1.5882053613510112e+61 3.5309331780841536e+63 1.5655163033958892e+63
13747539349728.273 8.535014528852476e+17 3.678526029235825e+18
2.3392983963847338e-11 7.500582237164971e-06 1.7149441369782873e-05
1.322754032349451e+18 24494745.48627819 469906.86960277177
1.1507635580772591e+20 1.3412883374328603e+31 1.4862906265854023e+29
417474338.8166057 0.0005391881963386104 970.5404089968766
2100310754.242473 79

,Target,Predição
0,Passive,Promoter
1,Detractor,Detractor
2,Promoter,Promoter
3,Passive,Passive
4,Passive,Passive
...,...,...
1075,Passive,Promoter
1076,Passive,Passive
1077,Detractor,Passive
1078,Detractor,Detractor


## Análise de Acertos e Erros do Classificador

Após rodarmos o classificador Naive-Bayes na **base de testes**, realizamos uma avaliação quantitativa dos resultados, comparando os rótulos reais (`Target`) com as predições do modelo (`Predição`).

### **Etapas do Cálculo**
1. **Filtragem dos Resultados por Categoria**
   - O DataFrame `df_resultado` é segmentado em três grupos: *Passive*, *Promoter* e *Detractor*.
   - Cada conjunto contém as avaliações reais e as predições do modelo.

2. **Contabilização de Acertos e Erros**
   - Para cada categoria, verificamos se a predição do modelo **coincide** com o rótulo verdadeiro.
   - Se coincidir → **Incrementamos o contador de acertos**.
   - Se for classificado incorretamente → **Incrementamos o contador de erros**.

### **Resultados Obtidos**
| Classe      | Acertos | Erros |
|------------|---------|-------|
| **Passive**  | 217     | 155   |
| **Promoter** | 225     | 116   |
| **Detractor** | 269     | 98    |

### **Interpretação**
- O classificador obteve **melhor desempenho na classe *Detractor***, possivelmente devido a palavras mais características que distinguem esse perfil.
- A classe **Passive apresentou o maior número de erros**, sugerindo que pode ter características **menos distintas**, tornando mais difícil para o modelo diferenciar de outras categorias.
- O desempenho geral pode ser aprimorado com ajustes como:
  - **Uso de bigramas** para capturar relações entre palavras.
  - **Remoção de palavras irrelevantes (stopwords avançado)**.
  - **Ponderação da frequência das palavras** para evitar viés excessivo por termos muito comuns.

Esses resultados servirão de base para calcular métricas detalhadas como **acurácia, precisão, recall e matriz de confusão** para uma análise mais aprofundada da performance do modelo.


In [106]:

Passivo = df_resultado.loc[df_resultado["Target"] == "Passive", "Predição"]
Promoter = df_resultado.loc[df_resultado["Target"] == "Promoter", "Predição"]
Detractor = df_resultado.loc[df_resultado["Target"] == "Detractor", "Predição"]

acerto_passivo = 0
erro_passivo = 0
acerto_promoter = 0
erro_promoter = 0
acerto_detrator = 0
erro_detrator = 0

for idx, predicao in Passivo.items():
    if predicao == "Passive":
        acerto_passivo += 1
    else:
        erro_passivo += 1

for idx, predicao in Promoter.items():
    if predicao == "Promoter":
        acerto_promoter += 1
    else:
        erro_promoter += 1

for idx, predicao in Detractor.items():
    if predicao == "Detractor":
        acerto_detrator += 1
    else:
        erro_detrator += 1

print(f"Passivo: {acerto_passivo} acertos, {erro_passivo} erros")
print(f"Promoter: {acerto_promoter} acertos, {erro_promoter} erros")
print(f"Detractor: {acerto_detrator} acertos, {erro_detrator} erros")


Passivo: 217 acertos, 155 erros
Promoter: 225 acertos, 116 erros
Detractor: 269 acertos, 98 erros


___
## Análise Qualitativa da Performance do Classificador

**Criando a Acurácia dos Preditores**



Nesta seção, avaliamos a **acurácia do classificador Naive-Bayes** para cada uma das categorias (*Passive*, *Detractor* e *Promoter*). 

### **Resultados Obtidos**
| Classe      | Acurácia (%) |
|------------|-------------|
| **Passive**  | 58.33%      |
| **Detractor** | 73.30%      |
| **Promoter**  | 65.98%      |
| **Média Geral** | **65.87%** |

- O **classificador teve melhor desempenho na classe *Detractor***, com 73.30% de acurácia. Isso sugere que esse grupo possui padrões mais distintos, facilitando a diferenciação.  
- A classe **Passive apresentou o pior desempenho (58.33%)**, indicando que há um maior número de confusões com outras categorias.  
- A **acurácia global média** ficou em torno de **65.87%**, o que demonstra um desempenho aceitável, mas com margem para melhorias.

In [107]:
acur_passivo = (acerto_passivo/(acerto_passivo+erro_passivo))*100
acur_detractor = (acerto_detrator/(acerto_detrator+erro_detrator))*100
acur_promoter = (acerto_promoter/(acerto_promoter+erro_promoter))*100
acur_total = ((acur_promoter+acur_detractor+acur_passivo)/3)
erros_totais = (erro_passivo+erro_detrator+erro_promoter)
acertos_totais = (acerto_passivo+acerto_detrator+acerto_promoter)
erros_totais = (erro_passivo+erro_detrator+erro_promoter)
acertos_totais = (acerto_passivo+acerto_detrator+acerto_promoter)


print(f"O total de acertos do classicador das Targets foi de {acertos_totais}")
print()
print(f"O total de erros do classificador de erros das Targets foi de {erros_totais}")
print()
print(f"A acurácia do classificador da Target Passive é de {acur_passivo:.2f}%")
print()
print(f"A acurácia do classificador da Target Detractor é de {acur_detractor:.2f}%")
print()
print(f"A acurácia do classificador da Target Promoter é de {acur_promoter:.2f}%")
print()



O total de acertos do classicador das Targets foi de 711

O total de erros do classificador de erros das Targets foi de 369

A acurácia do classificador da Target Passive é de 58.33%

A acurácia do classificador da Target Detractor é de 73.30%

A acurácia do classificador da Target Promoter é de 65.98%



Para obter uma métrica única que represente o desempenho global do classificador, calculamos a **acurácia média** entre as três categorias (*Passive*, *Detractor* e *Promoter*)

### **Resultado**
- **Acurácia Global do Classificador:** **65.87%**

In [108]:
acur_total = ((acur_promoter+acur_detractor+acur_passivo)/3)


print(f"A acurácia do classificador de Target foi de {acur_total:.2f}%")

A acurácia do classificador de Target foi de 65.87%


**Análise da quantidade de palavras nas frases de cada Target**

Realizamos uma análise estatística das palavras contidas nas avaliações da **base de testes**, segmentando-as por categoria (*Passive*, *Detractor* e *Promoter*). O objetivo é compreender melhor a distribuição do vocabulário em cada classe.

### **Processo**
1. **Filtragem das Avaliações**
   - Extraímos todas as avaliações marcadas como *Passive* na base de testes.
   - Convertamos as frases em uma **lista de textos**.

2. **Limpeza e Processamento do Texto**
   - Concatenamos todas as frases para formar um único corpo de texto.
   - Aplicamos a função `cleanup()` para remover caracteres especiais e converter para letras minúsculas.
   - **Tokenizamos** (separamos) as palavras.

3. **Cálculo de Estatísticas**
   - Criamos uma estrutura `pd.Series` para contar a **frequência de palavras** dentro das avaliações passivas.
   - Calculamos o **total de palavras** utilizadas nesse grupo.

### **Aplicação**
- Esse mesmo processo foi repetido para os rótulos **Detractor** e **Promoter**, permitindo comparar **o tamanho médio das avaliações**, a **riqueza do vocabulário** e a **presença de palavras mais frequentes** em cada classe.
- Essas informações auxiliam na melhoria do modelo Naive-Bayes, ajudando a identificar possíveis padrões distintos entre as categorias.

In [109]:
#PASSIVO

dic_passivos_test = test.loc[test["Target"]=="Passive","Review"]
#Transformando em Lista
frases_passivos_test = dic_passivos_test.tolist()

#Juntando as frases,Limpando (caracteres especiais, e diminuindo as letras)
frases_passivos_test = " ".join(frases_passivos_test)
frases_passivos_test = cleanup(frases_passivos_test).lower()
frases_passivos_test = frases_passivos_test.split()

#Probabilidade de ser Passivo
qtd_palavras_passivos_test = pd.Series(frases_passivos_test).value_counts()
qtd_palavras_passivos_test = qtd_palavras_passivos_test.sum()


In [110]:
#DETRACTOR


dic_detrator_test = test.loc[test["Target"]=="Detractor","Review"]

#Transformando em Lista
frases_detractor_test = dic_detrator_test.tolist()
#Juntando as frases,Limpando (caracteres especiais, e diminuindo as letras)
frases_detractor_test = " ".join(frases_detractor_test)
frases_detractor_test = cleanup(frases_detractor_test).lower()
frases_detractor_test = frases_detractor_test.split()

#Probabilidade de ser Detrator
qtd_palavras_detrator_test = pd.Series(frases_detractor_test).value_counts()
qtd_palavras_detrator_test = qtd_palavras_detrator_test.sum()



In [111]:
#PROMOTER

dic_promoter_test = test.loc[test["Target"]=="Promoter","Review"]

#Transformando em Lista
frases_promoter_test = dic_promoter_test.tolist()

#Juntando as frases,Limpando (caracteres especiais, e diminuindo as letras)
frases_promoter_test = " ".join(frases_promoter_test)
frases_promoter_test = cleanup(frases_promoter_test).lower()
frases_promoter_test = frases_promoter_test.split()

#Probabilidade de ser Detrator
qtd_palavras_promoter_test = pd.Series(frases_promoter_test).value_counts()
qtd_palavras_promoter_test = qtd_palavras_promoter_test.sum()

In [112]:
print(f"A quantidade de palavras de cada frases de Passivos é {qtd_palavras_passivos_test}")
print()
print(f"A quantidade de frases de Detrector é {qtd_palavras_detrator_test}")
print()
print(f"A quantidade de frases de Promoter é {qtd_palavras_promoter_test}")
print()



A quantidade de palavras de cada frases de Passivos é 49898

A quantidade de frases de Detrector é 55514

A quantidade de frases de Promoter é 37523



___
## Qualidade do Classificador a partir de novas separações das mensagens entre Treinamento e Teste

Caso for fazer esse item do Projeto

Refazendo a Limpeza dos textos


## Qualidade do Classificador a partir de Novas Separações das Mensagens entre Treinamento e Teste

### **Objetivo**
Nesta etapa, analisamos o impacto da **redivisão aleatória** entre treinamento e teste na performance do classificador. Isso nos ajuda a entender **quão estável** é o modelo em diferentes amostragens de dados.

### **Refazendo a Limpeza dos Textos**
Para garantir que os textos estejam bem preparados para análise, implementamos uma nova função de limpeza `cleanup2(text)`, que inclui:

1. **Separação das palavras** para processar individualmente cada termo.  
2. **Remoção de caracteres indesejados** como pontuações, números e símbolos (`!@#$%&*{}[];.,-0123456789`).  
3. **Tratamento de URLs e códigos**:
   - **Remove links (https://... ou http://...)** para evitar interferência na análise.  
   - **Remove sequências numéricas** que podem ser códigos ou IDs irrelevantes.  
4. **Conversão para minúsculas**, garantindo padronização no processamento de texto.  

### **Importância da Nova Limpeza**
- A remoção de **URLs e códigos numéricos** reduz o ruído nos dados e melhora a qualidade das predições.  
- Essa nova abordagem permite que a separação entre **treinamento e teste** seja mais eficiente, garantindo que o classificador Naive-Bayes trabalhe com dados melhor estruturados.  

Com essa reformulação da limpeza, o próximo passo é **repetir o processo de treinamento e avaliação do modelo em diferentes divisões dos dados** para analisar sua variação de desempenho.


In [113]:
def cleanup2(text):
    sep = text.split()
    indesejaveis = '!@#$%^&*()[]}{",;.-0123456789'
    textOut = ''
    for palavra in sep:
        word = ''
        if 'https://' not in palavra:
            if '/' in palavra:
                palavras = palavra.split('/')
                for i in palavras:
                    textOut += i
                    textOut += ' '
                word = ''
            else:
                for caractere in palavra:
                    if 'https://' in palavra:
                        word = ''
                        break # sumindo com urls
                    elif caractere in '0123456789':
                        word = ''
                        break # sumindo com códigos
                    elif caractere not in indesejaveis: 
                        word += caractere.lower()
        if word != '':
            textOut += word
            textOut += ' '
    return textOut



## Descobrindo a Quantidade Total de Palavras

Nesta etapa, extraímos a quantidade total de palavras presentes nas avaliações do conjunto de **treinamento**. O objetivo é analisar a distribuição do vocabulário e sua representatividade no modelo.

### **Processo de Extração e Limpeza**
1. **Coletamos todas as avaliações** da base de treinamento (`Review`).  
2. **Transformamos em uma única string** para facilitar o processamento.  
3. **Aplicamos a nova função de limpeza `cleanup2()`**, que remove caracteres indesejados, links e códigos numéricos.  
4. **Convertamos para minúsculas** para evitar distinções entre palavras idênticas com diferentes capitalizações.  
5. **Tokenizamos** (separamos) as palavras e armazenamos em um `pd.Series` para análise.  
6. **Contamos a frequência de cada palavra** e calculamos o total de termos.

### **Importância dessa Análise**
- Permite identificar **quais palavras são mais frequentes** dentro do conjunto de treinamento.  
- Ajuda a **compreender o vocabulário predominante**, que será usado no modelo Naive-Bayes.  
- Facilita a **comparação entre diferentes divisões de treinamento e teste**, permitindo verificar se a base usada é suficientemente representativa para todas as categorias (*Detractor, Promoter e Passive*).

Essa análise é essencial para garantir que o modelo esteja bem treinado e preparado para classificar novas avaliações com maior precisão.


In [114]:
#Descobrindo a quantidade total de palavras
dic_total2 = train.loc[:, "Review"]  
palavras_totais2 = dic_total2.tolist()  
palavras_totais2 = " ".join(palavras_totais2)  
palavras_totais2 = cleanup2(palavras_totais2).lower()
palavras_totais2 = palavras_totais2.split()  
palavras_totais2 = pd.Series(palavras_totais2)  
qtd_palavras_totais2 = palavras_totais2.value_counts()  
qtd_palavras_totais2 = qtd_palavras_totais2.sum()  

Criação de um StopWords

In [115]:
# # determinando palvras mais frequentes para o stopwords
# freq_rel_palvras_totais = palavras_totais2.value_counts(True)
# # limite = len(dic_total2.tolist())*0.015 # criando um limite de aparições de até 1,5% da base de dados

# #TENTATIVA DE CRIAR O STOPWORDS DAS PALAVRAS MAIS FREQUENTES

# # for word in freq_rel_palvras_totais:
# #     if word > limite:

# #         stopwords = set([word]) #stopwords

# # print(stopwords)
# # def tirando_stopwords(comment):
# #     return ' '.join([word.lower() for word in comment.split() if word.lower() not in stopwords])

# print(freq_rel_palvras_totais)

## Análise das Palavras por Categoria e Criação do Vocabulário

### **Processamento das Avaliações por Categoria**
Processamos separadamente as avaliações de cada **target** (*Passive, Detractor e Promoter*), garantindo que os textos sejam adequadamente tratados antes de alimentar o modelo Naive-Bayes.

#### **1. Extração e Limpeza dos Textos**
- **Filtramos** os textos pertencentes a cada classe.
- **Convertamos para uma lista** para facilitar o processamento.
- **Concatenamos os textos**, criando uma única string para cada categoria.
- **Aplicamos a limpeza** com `cleanup2()`, removendo caracteres especiais, stopwords e convertendo para minúsculas.
- **Tokenizamos** os textos, separando as palavras para análise.

#### **2. Cálculo da Probabilidade de Cada Classe**
- **Contamos a frequência de cada palavra** nas avaliações de cada categoria.
- **Calculamos a probabilidade a priori** de cada classe (`Prob_Passivo`, `Prob_Detrator`, `Prob_Promoter`), dividindo o total de palavras em cada categoria pelo total geral.

#### **3. Frequência Relativa das Palavras**
- Calculamos:
  - **Frequência relativa** (proporção de cada palavra dentro de cada categoria).
  - **Frequência absoluta** (quantidade total de ocorrências de cada palavra).

---

### **Criação do Vocabulário Global**
Após a análise individual de cada categoria, construímos um **vocabulário global**, contendo **todas as palavras únicas** encontradas no conjunto de treinamento.

1. **Criamos uma lista vazia (`vocabulario2`)**.
2. **Percorremos todas as palavras do dataset** e adicionamos ao vocabulário apenas aquelas que ainda não estavam presentes.
3. **Armazenamos a quantidade total de palavras únicas**, que será usada no cálculo de probabilidades do modelo Naive-Bayes.

---

### **Importância desta Etapa**
1. Permite ao modelo identificar **padrões específicos de cada classe**, auxiliando na diferenciação entre avaliações **positivas, neutras e negativas**.  
2. Garante que o modelo trabalhe com um **vocabulário limpo e padronizado**, reduzindo ruídos que poderiam comprometer a performance.  
3. Ajuda a evitar **viés no modelo**, distribuindo corretamente as palavras entre as categorias.

Com esse processamento, avançamos para a **fase de treinamento do classificador Naive-Bayes**, utilizando os dados devidamente preparados.


In [116]:
#PASSIVO

dic_passivos2 = train.loc[train["Target"]=="Passive","Review"]
#Transformando em Lista
frases_passivos2 = dic_passivos2.tolist()

#Juntando as frases,Limpando (caracteres especiais, e diminuindo as letras)
frases_passivos2 = " ".join(frases_passivos2)
#frases_passivos2= tirando_stopwords(frases_passivos2)
frases_passivos2 = cleanup2(frases_passivos2).lower()
frases_passivos2 = frases_passivos2.split()

#Probabilidade de ser Passivo
qtd_palavras_passivos2 = pd.Series(frases_passivos2).value_counts()
qtd_palavras_passivos2 = qtd_palavras_passivos2.sum()

Prob_Passivo2 = qtd_palavras_passivos2/qtd_palavras_totais2

#Criando Frequencia Relativa das Palavras

freq_rel_passivos2 = pd.Series(frases_passivos2).value_counts(True)
freq_abs_passivos2 = pd.Series(frases_passivos2).value_counts()


#DETRACTOR

dic_detrator2 = train.loc[train["Target"]=="Detractor","Review"]

#Transformando em Lista
frases_detractor2 = dic_detrator2.tolist()
#Juntando as frases,Limpando (caracteres especiais, e diminuindo as letras)
frases_detractor2 = " ".join(frases_detractor2)
#frases_detractor2= tirando_stopwords(frases_detractor2)
frases_detractor2 = cleanup2(frases_detractor2).lower()
frases_detractor2 = frases_detractor2.split()

#Probabilidade de ser Detrator
qtd_palavras_detrator2 = pd.Series(frases_detractor2).value_counts()
qtd_palavras_detrator2 = qtd_palavras_detrator2.sum()

Prob_Detrator2 = qtd_palavras_detrator2/qtd_palavras_totais2

#Criando Frequencia Relativa das Palavras

freq_rel_detrator2 = pd.Series(frases_detractor2).value_counts(True)
freq_abs_detrator2 = pd.Series(frases_detractor2).value_counts()


#PROMOTER

dic_promoter2 = train.loc[train["Target"]=="Promoter","Review"]

#Transformando em Lista
frases_promoter2 = dic_promoter2.tolist()

#Juntando as frases,Limpando (caracteres especiais, e diminuindo as letras)
frases_promoter2 = " ".join(frases_promoter2)
#frases_promoter2= tirando_stopwords(frases_promoter2)
frases_promoter2 = cleanup2(frases_promoter2).lower()
frases_promoter2 = frases_promoter2.split()

#Probabilidade de ser Detrator
qtd_palavras_promoter2 = pd.Series(frases_promoter2).value_counts()
qtd_palavras_promoter2 = qtd_palavras_promoter2.sum()


Prob_Promoter2 = qtd_palavras_promoter2/qtd_palavras_totais2

#Criando Frequencia Relativa das Palavras

freq_rel_promoter2 = pd.Series(frases_promoter2).value_counts(True)
freq_abs_promoter2 = pd.Series(frases_promoter2).value_counts()



#Criando VOCABULARIO
#Adicionando todas as palavras presentes nas frases uma unica vez:
vocabulario2 = []

for i in palavras_totais2:
    if i not in vocabulario2:
        vocabulario2.append(i)


qtd_vocabulario2 = len(vocabulario2)

 Tentativa de criar proporções entre as frases 

In [117]:
# # Na nova iteração iremos fazer a multiplicação com base na proporção de frases que cada um tem

# # Total de frases em cada cliente
# total_detractor = len(frases_detractor2)
# total_promoter = len(frases_promoter2)
# total_passivo = len(frases_passivos2)

# # Total de frases
# total_de_frases = total_detractor + total_promoter + total_passivo

# # Criação das proporções
# prop_detrator = total_detractor / total_de_frases
# prop_passivo = total_passivo / total_de_frases
# prop_promoter = total_promoter / total_de_frases

# # Ajuste baseado no inverso da proporção
# peso_detrator = 1 / prop_detrator
# peso_passivo = 1 / prop_passivo
# peso_promoter = 1 / prop_promoter

# # Normalizar os pesos para que não sejam desbalanceados
# soma_pesos = peso_detrator + peso_passivo + peso_promoter
# peso_detrator /= soma_pesos
# peso_passivo /= soma_pesos
# peso_promoter /= soma_pesos

# # Exibir os pesos ajustados
# print("Peso Detrator:", peso_detrator)
# print("Peso Passivo:", peso_passivo)
# print("Peso Promoter:", peso_promoter)




**Verificando Performance do Classificador**

## Verificando a Performance do Classificador com Novas Separações

### **Objetivo**
Nesta etapa, testamos novamente o **classificador Naive-Bayes** com uma nova distribuição dos dados de teste. O objetivo é avaliar a **consistência e estabilidade do modelo**, garantindo que ele mantenha um desempenho robusto em diferentes amostragens.

---

### **Processo de Teste**
1. **Extração dos Dados de Teste**
   - Coletamos os rótulos reais (`Target_test2`) e as avaliações (`Review_test2`).
   - Criamos um dicionário (`dic_final2`) para armazenar os resultados das predições.

2. **Limpeza e Tokenização**
   - Aplicamos a função `cleanup2()` para remover caracteres indesejados e padronizar os textos.
   - Convertamos as frases em listas de palavras individuais.

3. **Cálculo das Probabilidades**
   - Inicializamos as probabilidades de cada classe (`Detractor`, `Promoter`, `Passive`) com valor 1.
   - Para cada palavra do texto:
     - Se a palavra estiver presente no vocabulário da classe, aplicamos a **fórmula de Naive-Bayes** com **suavização de Laplace**.
     - Se a palavra for desconhecida, aplicamos um valor suavizado para evitar probabilidades nulas.

4. **Determinação da Classe**
   - Comparando as probabilidades calculadas, selecionamos a **classe com maior probabilidade**.
   - A predição final (`final2`) é armazenada no dicionário.

5. **Criação do DataFrame de Resultados**
   - Convertamos `dic_final2` para um **DataFrame pandas**, permitindo visualizar as predições ao lado dos rótulos verdadeiros.

---

### **Importância dessa Etapa**
1. **Valida a estabilidade do modelo**, garantindo que ele funcione bem em diferentes divisões dos dados.  
2. **Permite identificar variações de desempenho**, indicando se há necessidade de ajustes no pré-processamento.  
3. **Prepara o modelo para análise de métricas**, como **acurácia, precisão, recall e matriz de confusão**.  

Com essa nova verificação, seguimos para a avaliação quantitativa do desempenho do classificador, comparando seus acertos e erros.


In [118]:
dic_final2 = {}
Target_test2 = test.loc[:, "Target"]
Review_test2 = test.loc[:, "Review"]

for i in range(len(test)):
    tipo2 = Target_test2.iloc[i]  
    frase2 = cleanup2(Review_test2.iloc[i]).lower().strip()
    frase2 = " ".join(frase2.split())  

    probDetratordadoFrase2 = 1
    probPassivodadoFrase2 = 1
    probPromoterdadoFrase2 = 1
    for palavra2 in frase2.split():
        palavra2 = palavra2.lower().strip()

        if palavra2 in frases_promoter2:
            probPromoterdadoFrase2 *= (freq_abs_promoter2[palavra2]+ 1)/(qtd_palavras_promoter2 + qtd_vocabulario2) * 1000 
        else:
            probPromoterdadoFrase2 *= (1/(qtd_palavras_promoter2 + qtd_vocabulario2))  * 1000  

        if palavra2 in frases_detractor2:
            probDetratordadoFrase2 *= (freq_abs_detrator2[palavra2]+ 1)/(qtd_palavras_detrator2 + qtd_vocabulario2) * 1000 
        else:
            probDetratordadoFrase2 *= (1/(qtd_palavras_detrator2 + qtd_vocabulario2)) * 1000 

        if palavra2 in frases_passivos2:
            probPassivodadoFrase2 *= (freq_abs_passivos2[palavra2]+ 1)/(qtd_palavras_passivos2 + qtd_vocabulario2)   * 1000 
        else:
            probPassivodadoFrase2 *= (1/(qtd_palavras_passivos2 + qtd_vocabulario2))  * 1000 


    #maior2 = max(probDetratordadoFrase2*(1+peso_detrator), probPromoterdadoFrase2*(1+peso_promoter), probPassivodadoFrase2*(1+peso_passivo))
    
    maior2 = max(probDetratordadoFrase2, probPromoterdadoFrase2,probPassivodadoFrase2)
    print(probDetratordadoFrase2, probPromoterdadoFrase2, probPassivodadoFrase2)

    if maior2 == probDetratordadoFrase2:
        final2 = "Detractor"
    elif maior2 == probPromoterdadoFrase2:
        final2 = "Promoter"
    else:
        final2 = "Passive"

    dic_final2[i] = (tipo2, final2)

import pandas as pd
df_resultado2 = pd.DataFrame.from_dict(dic_final2, orient="index", columns=["Target", "Predição"])

df_resultado2


394927389.8117773 1.6324948278883482e+16 129306485056986.67
2.879320877441464e+29 1.2276434080197265e+20 1.406224685771661e+28
82007033.24675193 2238556601841473.0 1051460680161930.6
19287.546659732092 177146158.53111547 15210818939.098475
1.077917126388837e+16 3.861258766883535e+21 4.6546364464010176e+23
12925667372221.2 3.007703091854564e+20 3.4340994342084787e+18
544816.9201935816 929310459.611756 389672521.5118463
6.226527940882713e+19 5.456852718808067e+21 1.396517086785796e+29
1.201843538884879e+18 81365602055389.84 2.500539789316603e+18
515793817.25047916 3347592440.6511965 10413022354.514986
1.585279293549518e+66 7.775329850385568e+68 2.0626335517241638e+68
3043439931344075.5 6.883245109136752e+19 4.094032368436582e+20
4.194661930271409e-09 0.0032097075728035377 0.0037733457788065015
2.90613923395633e+24 26277327178793.297 634740807989.5231
5.5917969703229703e+23 8.536867070058057e+34 2.7578093377938916e+32
2780526872740.2837 34.70212389251925 46115004.221381076
384185653570247

,Target,Predição
0,Passive,Promoter
1,Detractor,Detractor
2,Promoter,Promoter
3,Passive,Passive
4,Passive,Passive
...,...,...
1075,Passive,Promoter
1076,Passive,Passive
1077,Detractor,Passive
1078,Detractor,Detractor


## Avaliação de Acertos e Erros do Classificador

Após a execução do modelo Naive-Bayes sobre a nova base de testes, analisamos a quantidade de **acertos e erros** para cada categoria (*Passive*, *Promoter* e *Detractor*).

---

### **Etapas da Análise**
1. **Filtragem dos Resultados por Categoria**
   - O DataFrame `df_resultado2` é segmentado nas três classes (*Passive*, *Promoter* e *Detractor*).
   - Extraímos os rótulos verdadeiros e as predições associadas.

2. **Contagem de Acertos e Erros**
   - Para cada classe, verificamos se a predição do modelo **coincide** com o rótulo verdadeiro.
   - Se coincidir → **Incrementamos o contador de acertos**.
   - Se for classificado incorretamente → **Incrementamos o contador de erros**.

---

### **Resultados Obtidos**
| Classe      | Acertos | Erros |
|------------|---------|-------|
| **Passive**  | 221     | 151   |
| **Promoter** | 224     | 117   |
| **Detractor** | 267     | 100   |

---

### **Interpretação**
- O classificador obteve **melhor desempenho na classe *Detractor***, com 267 acertos e 100 erros, indicando que essa categoria possui padrões mais distintos no vocabulário.
- A classe **Passive apresentou o maior número de erros**, sugerindo que pode ter características **menos diferenciadas**, dificultando sua identificação pelo modelo.
- A **classe Promoter teve um desempenho intermediário**, com 224 acertos e 117 erros.

---


In [119]:

Passivo2 = df_resultado2.loc[df_resultado2["Target"] == "Passive", "Predição"]
Promoter2 = df_resultado2.loc[df_resultado2["Target"] == "Promoter", "Predição"]
Detractor2 = df_resultado2.loc[df_resultado2["Target"] == "Detractor", "Predição"]

acerto_passivo2 = 0
erro_passivo2 = 0
acerto_promoter2 = 0
erro_promoter2 = 0
acerto_detrator2 = 0
erro_detrator2 = 0

for idx2, predicao2 in Passivo2.items():
    if predicao2 == "Passive":
        acerto_passivo2 += 1
    else:
        erro_passivo2 += 1

for idx2, predicao2 in Promoter2.items():
    if predicao2 == "Promoter":
        acerto_promoter2 += 1
    else:
        erro_promoter2 += 1

for idx2, predicao2 in Detractor2.items():
    if predicao2 == "Detractor":
        acerto_detrator2 += 1
    else:
        erro_detrator2 += 1


print(f"Passivo: {acerto_passivo2} acertos, {erro_passivo2} erros")
print(f"Promoter: {acerto_promoter2} acertos, {erro_promoter2} erros")
print(f"Detractor: {acerto_detrator2} acertos, {erro_detrator2} erros")


Passivo: 221 acertos, 151 erros
Promoter: 224 acertos, 117 erros
Detractor: 267 acertos, 100 erros


## Avaliação da Acurácia do Classificador

Calculamos a **acurácia do modelo Naive-Bayes** para cada uma das categorias (*Passive*, *Detractor* e *Promoter*), além da **acurácia geral**.

### **Resultados Obtidos**
| Classe      | Acurácia (%) |
|------------|-------------|
| **Passive**  | 59.41%      |
| **Detractor** | 72.75%      |
| **Promoter**  | 65.69%      |
| **Média Geral** | **65.95%** |

- O classificador teve **melhor desempenho na classe *Detractor***, com **72.75%** de acurácia, sugerindo que essa categoria apresenta um vocabulário mais distinto e fácil de identificar.
- A classe **Passive teve o pior desempenho (59.41%)**, indicando que seu vocabulário pode ser mais similar ao de outras classes, tornando sua diferenciação mais difícil.
- A **acurácia global** do modelo ficou em **65.95%**, o que demonstra um desempenho razoável, mas com espaço para melhorias.

---



In [120]:
acur_passivo2 = (acerto_passivo2/(acerto_passivo2+erro_passivo2))*100
acur_detractor2 = (acerto_detrator2/(acerto_detrator2+erro_detrator2))*100
acur_promoter2 = (acerto_promoter2/(acerto_promoter2+erro_promoter2))*100
acur_total2 = ((acur_promoter2+acur_detractor2+acur_passivo2)/3)
erros_totais2 = (erro_passivo2+erro_detrator2+erro_promoter2)
acertos_totais2 = (acerto_passivo2+acerto_detrator2+acerto_promoter2)
erros_totais2 = (erro_passivo2+erro_detrator2+erro_promoter2)
acertos_totais2 = (acerto_passivo2+acerto_detrator2+acerto_promoter2)


print(f"O total de acertos do classicador das Targets foi de {acertos_totais2}")
print()
print(f"O total de erros do classificador de erros das Targets foi de erros totais {erros_totais2}")
print()
print(f"A acurácia do classificador da Target Passive é de {acur_passivo2:.2f}%")
print()
print(f"A acurácia do classificador da Target Detractor é de {acur_detractor2:.2f}%")
print()
print(f"A acurácia do classificador da Target Promoter é de {acur_promoter2:.2f}%")
print()

acur_total2 = ((acur_promoter2+acur_detractor2+acur_passivo2)/3)


print(f"A acurácia do classificador de Target foi de {acur_total2:.2f}%")

O total de acertos do classicador das Targets foi de 712

O total de erros do classificador de erros das Targets foi de erros totais 368

A acurácia do classificador da Target Passive é de 59.41%

A acurácia do classificador da Target Detractor é de 72.75%

A acurácia do classificador da Target Promoter é de 65.69%

A acurácia do classificador de Target foi de 65.95%


# **Considerações Finais**

## **Resumo das Etapas Realizadas**
O projeto seguiu um fluxo estruturado para a construção e avaliação de um **classificador Naive-Bayes** aplicado à classificação de avaliações (*reviews*). As principais etapas foram:

- **Criação das bases de teste e treino** *(Célula 60)*  
- **Implementação da função de limpeza (`cleanup`)** *(Célula 64)*  
- **Determinação da quantidade total de palavras no dataset** *(Célula 65)*  
- **Análise dos perfis: Passive, Detractor e Promoter** *(Células 66 a 69)*  
- **Criação do vocabulário com palavras únicas** *(Célula 83)*  
- **Aplicação da Suavização de Laplace** *(Célula 86)*  
- **Contagem de acertos e erros do modelo** *(Célula 71)*  
- **Cálculo da acurácia para cada classe e do modelo geral** *(Células 72 e 73)*  
- **Segunda iteração do modelo, separando novamente os dados de treino e teste** *(Células 78, 85, 23, 80 e 81)*  

---

## **Uso da Suavização de Laplace**
A **suavização de Laplace** foi essencial para evitar que palavras **desconhecidas pelo modelo** tivessem probabilidade zero, o que levaria a problemas na classificação. O método assegura que todas as palavras, mesmo as que aparecem **poucas vezes**, recebam uma probabilidade maior que zero, garantindo **melhor generalização do modelo**.

- Sem a suavização, o modelo interpretaria avaliações contendo palavras inéditas como **inclassificáveis**.  
- Com esse método, as probabilidades tendem a valores muito baixos, mas **nunca chegam a zero**, permitindo uma classificação mais precisa. *(Detalhes matemáticos na célula 86).*

---

## **Alterações Implementadas**
Para melhorar a precisão do classificador, aplicamos refinamentos nas seguintes áreas:

1. **Otimização da Função de Limpeza (`cleanup`)**  
   - Agora, a função considera casos onde palavras são separadas por `/`.  
   - Links e números passaram a ser ignorados.  

2. **Remoção de Stopwords Avançada**  
   - Implementamos um filtro para eliminar palavras com **alta frequência (>1.5%)** na base de treinamento, reduzindo ruído.

3. **Reestruturação da Atribuição de Pesos**  
   - Aplicamos pesos diferentes às frases com base no seu perfil de classificação.  
   - Durante essa etapa, identificamos um **viés na distribuição das frases**, que influenciava a classificação.  
   - No entanto, essa abordagem **reduziu a acurácia geral do modelo**, sugerindo a necessidade de um **balanceamento mais refinado**.

---



# **Conclusão e Possíveis Melhorias**

## **Conclusão**

Na **primeira iteração**, utilizamos apenas o **método de limpeza básico (`cleanup`)** fornecido pelo projeto, alcançando uma **acurácia de 65,87%**, com melhor desempenho na classificação da categoria **Detractor**.

Na **segunda iteração**, refinamos a **limpeza dos dados**, removendo palavras irrelevantes e criando uma **lista de stopwords** personalizada. Como palavras muito frequentes podem aparecer em diferentes contextos, eliminá-las ajudou a **reduzir a interferência de termos genéricos** na classificação. Além disso, aplicamos **pesos proporcionais** para compensar a **desigualdade na quantidade de frases por classe**.

Apesar dessas melhorias, os ajustes **reduziram a acurácia para 59%**, tornando a aplicação inviável. Assim, optamos por **manter a nova limpeza**, mas descartamos os pesos adicionais, finalizando o modelo com **acurácia de 65,95%**, valor próximo ao inicial, mas com **dados melhor organizados e pré-processados**.

Além disso, identificamos desafios específicos na **classificação automatizada de avaliações de clientes**:
- **Sentimentos extremos** são mais fáceis de classificar (*clientes muito satisfeitos ou muito insatisfeitos*).
- **Frases descritivas sem opinião clara** dificultam a categorização automática.
- O modelo **não interpreta contexto emocional**, resultando em classificações ambíguas em algumas situações.

---

## **Aperfeiçoamento**
Para alcançar melhores resultados e evoluir o projeto, algumas sugestões de aprimoramento incluem:

✅ **Otimização da Limpeza e Transformação de Texto**  
   - Implementar **stemming e lemmatization**, reduzindo variações morfológicas das palavras.  
   - Refinar a remoção de **stopwords** para manter apenas termos realmente relevantes.

✅ **Melhoria no Modelo Naive-Bayes**  
   - Testar a **inclusão de bigramas** para capturar melhor a relação entre palavras.  
   - Ajustar a **suavização de Laplace** e explorar outras técnicas de balanceamento de dados.  
   - Justificar matematicamente a escolha das fórmulas e apresentar referências sobre o método utilizado.

✅ **Aprofundamento na Análise de Resultados**  
   - Explicar **por que novas mensagens classificadas não devem ser usadas para re-treinamento do modelo**.  
   - Refletir criticamente sobre **erros do modelo**, identificando padrões e possíveis soluções.  
   - Comparar diferentes cenários para o Naive-Bayes no contexto do projeto (exemplo: aplicação do modelo em **filtro de spam**).

✅ **Documentação Clara e Detalhada**  
   - Explicar cada etapa do processo, incluindo **decisões de modelagem e transformações de dados**.  
   - Indicar **como implementar melhorias**, fornecendo materiais de pesquisa para referências.

Com essas estratégias, o modelo poderá **atingir uma acurácia superior a 70%**, tornando-se mais confiável e aplicável em diferentes contextos. 


___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[Natural Language Processing (Part 17)-Laplacian Smoothing](https://medium.com/@Coursesteach/natural-language-processing-part-17-laplacian-smoothing-7d4be71d0ded) **Mais simples**